## EDA Statistical Analysis

### Objective : 
- Use Pyspark library
- Perform EDA Statistical analysis
    - Mean, Median , Standard deviation, Variance of Numerical columns
    - Outliers check
- Make generalize solution

### Import Libraries

In [1]:
import findspark
findspark.init()
import pyspark
findspark.find()

'D:\\PySpark\\spark-3.1.1-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
import seaborn as sns
import matplotlib

### Read Dataset

In [4]:
###Read dataset using Pyspark
### Observe first 5 rows
df = spark.read.csv(r'C:\Users\sayli\Desktop\Data_Science_practice\Apache Spark Udemy course\StudentsPerformance.csv',header=True, 
    mode="DROPMALFORMED")
df.show(5) 

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

### Data invistigate 

In [5]:
###Get the datatype of the dataframe
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: string (nullable = true)
 |-- reading score: string (nullable = true)
 |-- writing score: string (nullable = true)



In [6]:
### Convert the datatypes to get statistical significance 
df = df.withColumn('math score', df['math score'].cast('float'))
df = df.withColumn('reading score', df['reading score'].cast('float'))
df = df.withColumn('writing score', df['writing score'].cast('float'))
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: float (nullable = true)
 |-- reading score: float (nullable = true)
 |-- writing score: float (nullable = true)



In [7]:
### check for missing values
Dict_Null = {col: df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

{'gender': 0,
 'race/ethnicity': 0,
 'parental level of education': 0,
 'lunch': 0,
 'test preparation course': 0,
 'math score': 0,
 'reading score': 0,
 'writing score': 0}

### Ststitical Analysis

In [8]:
## check statistical significance 
df.describe('math score').show()

+-------+------------------+
|summary|        math score|
+-------+------------------+
|  count|              1000|
|   mean|            66.089|
| stddev|15.163080096009454|
|    min|               0.0|
|    max|             100.0|
+-------+------------------+



In [9]:
###Function to get median of the particular column
def get_median(col_name):
    median=df.approxQuantile(col_name, [0.5], 0.25)
    return median

In [10]:
get_median("math score")
#df.approxQuantile("math score", [0.5], 0.25)

[57.0]

In [11]:
get_median("reading score")

[59.0]

In [12]:
get_median("writing score")

[58.0]

In [13]:
### get median of all numerical columns
median = {c : df.approxQuantile(c, [0.5], 0.25) for c,d in zip(df.columns, df.dtypes) if d[1] == "float"}
median

{'math score': [57.0], 'reading score': [59.0], 'writing score': [58.0]}

In [14]:
## find mean of reading score column
df.agg({'reading score': 'mean'}).show()

+------------------+
|avg(reading score)|
+------------------+
|            69.169|
+------------------+



In [15]:
### get mean of all numerical columns
mean = {c : df.agg({c: 'mean'}).collect() for c,d in zip(df.columns, df.dtypes) if d[1] == "float"}
mean

{'math score': [Row(avg(math score)=66.089)],
 'reading score': [Row(avg(reading score)=69.169)],
 'writing score': [Row(avg(writing score)=68.054)]}

In [16]:
### get std_deviation of all numerical columns
std_deviation = {c : df.agg({c: 'stddev'}).collect() for c,d in zip(df.columns, df.dtypes) if d[1] == "float"}
std_deviation

{'math score': [Row(stddev(math score)=15.163080096009454)],
 'reading score': [Row(stddev(reading score)=14.600191937252223)],
 'writing score': [Row(stddev(writing score)=15.19565701086966)]}

In [17]:
### get variance of all numerical columns
variance = {c : df.agg({c: 'variance'}).collect() for c,d in zip(df.columns, df.dtypes) if d[1] == "float"}
variance

{'math score': [Row(variance(math score)=229.91899799799805)],
 'reading score': [Row(variance(reading score)=213.16560460460482)],
 'writing score': [Row(variance(writing score)=230.90799199199222)]}

In [18]:
### class to check statistical_analysis
class stat_analysis():
    
    def __init__(self, df):
        self.df = df
     
    def _calculate_median(self):
        median = {c : self.df.approxQuantile(c, [0.5], 0.25) for c,d in zip(self.df.columns, self.df.dtypes) if d[1] == "float"}
        return median
    
    def _calculate_mean(self):
        mean = {c : self.df.agg({c: 'mean'}).collect() for c,d in zip(self.df.columns, self.df.dtypes) if d[1] == "float"}
        return mean
    
    def _calculate_std_deviation(self):
        std_deviation = {c : self.df.agg({c: 'stddev'}).collect() for c,d in zip(self.df.columns, self.df.dtypes) if d[1] == "float"}
        return std_deviation
    

In [19]:
stat_analysis(df)._calculate_median()

{'math score': [57.0], 'reading score': [59.0], 'writing score': [58.0]}

### Outlier Check

In [20]:
bounds = {
    c: dict(
        zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c,d in zip(df.columns, df.dtypes) if d[1] == "float"
}
bounds

{'math score': {'q1': 57.0, 'q3': 77.0},
 'reading score': {'q1': 59.0, 'q3': 79.0},
 'writing score': {'q1': 57.0, 'q3': 79.0}}

In [21]:
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)
print(bounds)

{'math score': {'q1': 57.0, 'q3': 77.0, 'lower': 27.0, 'upper': 107.0}, 'reading score': {'q1': 59.0, 'q3': 79.0, 'lower': 29.0, 'upper': 109.0}, 'writing score': {'q1': 57.0, 'q3': 79.0, 'lower': 24.0, 'upper': 112.0}}


In [22]:
df1=df.select(
    "*",
    *[
        f.when(
            f.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c,d in zip(df.columns, df.dtypes) if d[1] == "float"
    ]
)
df1.show(5)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+--------------+-----------------+-----------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|math score_out|reading score_out|writing score_out|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+--------------+-----------------+-----------------+
|female|       group B|          bachelor's degree|    standard|                   none|      72.0|         72.0|         74.0|             0|                0|                0|
|female|       group C|               some college|    standard|              completed|      69.0|         90.0|         88.0|             0|                0|                0|
|female|       group B|            master's degree|    standard|                   none|      90.0|      

In [23]:
###Check the outliers using class and write all above mentioned functions 
class Outlier():

    def __init__(self, df):
        self.df = df


    def _calculate_bounds(self):
        bounds = {
            c: dict(
                zip(["q1", "q3"], self.df.approxQuantile(c, [0.25, 0.75], 0))
            )
            for c, d in zip(self.df.columns, self.df.dtypes) if d[1] in ["float", "int"]
        }

        for c in bounds:
            iqr = bounds[c]['q3'] - bounds[c]['q1']
            bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
            bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)

        return bounds


    def _flag_outliers_df(self):
        bounds = self._calculate_bounds()

        outliers_col = [
            f.when(
                ~f.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
                f.col(c)
            ).alias(c + '_outlier')
            for c in bounds]

        return self.df.select(*outliers_col)


    def show_outliers(self):

        outlier_df = self._flag_outliers_df()

        for outlier in outlier_df.columns:
            outlier_df.select(outlier).filter(f.col(outlier).isNotNull()).show()

In [24]:
Outlier(df).show_outliers()

+------------------+
|math score_outlier|
+------------------+
|              18.0|
|               0.0|
|              22.0|
|              24.0|
|              26.0|
|              19.0|
|              23.0|
|               8.0|
+------------------+

+---------------------+
|reading score_outlier|
+---------------------+
|                 17.0|
|                 26.0|
|                 28.0|
|                 23.0|
|                 24.0|
|                 24.0|
+---------------------+

+---------------------+
|writing score_outlier|
+---------------------+
|                 10.0|
|                 22.0|
|                 19.0|
|                 15.0|
|                 23.0|
+---------------------+



#### EDA Statistical analysis is performed using Pyspark on StudentsPerformance dataset. This analysis would provide basic understanding of Pyspark library and operations on dataframes using spark.
#### Gereralized solution is developed to perform statistical analysis on the dataset